# This software will:
    * Retweet any tweet
    * Schedule a new tweet
    * Edit a scheduled tweet
    

In [1]:
import config

consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret

# config.username
# config.password

from outboard_scrape import rand_user_agent

# Internets navigation
# from urllib.request import Request, urlopen

# Scraping tool
from bs4 import BeautifulSoup

# Import Tweepy
import tweepy

import logging
import os
import pandas as pd
import pickle
import re
import time
import requests


In [43]:

logger = logging.getLogger()

def create_api():


# Set Authentication Credentials
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

# Create API object
    api = tweepy.API(auth, wait_on_rate_limit=True,
                        wait_on_rate_limit_notify=True)

# Create API Object
    api = tweepy.API(auth)

    try:
        api.verify_credentials()
        print("Authentication OK")
    except Exception as e:
        logger.err0r("Error creating API", exc_info=True)
        raise e
#         print("Error during authentication")
    logger.info("API created")
    return api

create_api()

Authentication OK


In [22]:
# url = str(input("Please enter the url of the tweet you would like retweeted."))
url = 'https://twitter.com/KrakoaWelcomes/status/1268958023277518851'
id = re.findall(r'\d{19}$', url)
id[0]

'1268958023277518851'

In [23]:
def retweet():
    
    # Set Authentication Credentials
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

# Create API object
    api = tweepy.API(auth, wait_on_rate_limit=True,
                        wait_on_rate_limit_notify=True)
    api = tweepy.API(auth)
    url = str(input("Please enter the url of the tweet you would like retweeted."))
    id = re.findall(r'\d{19}$', url)
    api.retweet(id[0])
    print('Your tweet has been successfully retweeted!')

retweet()
    
#    schedule post time
#  edit post time (delete scheduled post and create new scheduled tweet)

Please enter the url of the tweet you would like retweeted.https://twitter.com/KrakoaWelcomes/status/1268958023277518851
Your tweet has been successfully retweeted!


In [13]:
# Collect the usernames of all followers
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


my_followers = []
for follower in tweepy.Cursor(api.followers, screen_name='KrakoaWelcomes').items():
    my_followers.append(follower.screen_name)
    time.sleep(1)

Rate limit reached. Sleeping for: 483
Rate limit reached. Sleeping for: 591
Rate limit reached. Sleeping for: 590
Rate limit reached. Sleeping for: 591
Rate limit reached. Sleeping for: 591


## With the list of usernames:
## 1. plug it into the photo url 
## 2. scrape the actual file url from html code 
## 3. download the photo from the file url

In [20]:
my_followers.to_csv('krakoa_followers.csv')

In [23]:
my_followers = pd.read_csv('krakoa_followers.csv', index_col = 0)

In [40]:
my_followers = my_followers.rename(columns={'0': 'followers'})

In [42]:
my_followers['followers'][0]

'SCGWalker'

In [6]:
for follower in my_followers:
    print(follower)

Unnamed: 0
0


In [53]:
import re
pattern = re.compile("h.*g")
result = pattern.findall('<img alt="" draggable="true" src="https://pbs.twimg.com/profile_images/893494961366118400/Bi95bJ3r_400x400.jpg" class="css-9pa8cd">')
print(result)


['https://pbs.twimg.com/profile_images/893494961366118400/Bi95bJ3r_400x400.jpg']


In [61]:
!pip install selenium

In [102]:
print('https://twitter.com/' + follower + '/photo')

https://twitter.com/JD_ComicNerd89/photo


In [109]:
my_followers[0:3]

['SCGWalker', 'DoXBookChat', 'azizdraim']

In [110]:
picture_urls[0:3]

[['https://pbs.twimg.com/profile_images/1215514540021932032/s1W81Kan_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/1232298147864510464/pZNpQVhR_400x400.jpg'],
 []]

In [114]:
result = None

In [122]:
my_followers[0]

'SCGWalker'

In [125]:
new_picture_urls = []

options = webdriver.ChromeOptions()
options.add_argument('headless')
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
driver = webdriver.Chrome(options=options, desired_capabilities=capa)
driver.set_window_size(1440,900)
driver.get('https://twitter.com/' + my_followers[0] + '/photo')
# print('getting user #' + str(i) + "'s photo url!")
# time.sleep(15)
   
plain_text = driver.page_source
new_soup = BeautifulSoup(plain_text, 'lxml')
new_soup
# pattern = re.compile('draggable="true" src="(.*).+?(?=\/>)')
# result = pattern.findall(str(new_soup))
# print(result)
# result = result[0]
# print(result)
# new_pattern = re.compile('(^.*.jpg)')
# new_result = new_pattern.findall(str(result))
# print(new_result)
# new_picture_urls.append(new_result)
# i+=1

<html><head></head><body></body></html>

In [106]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

picture_urls = []
i = 1

# source_code = requests.get('http://fantasy.espn.com/basketball/league/standings?leagueId=633975')
for follower in my_followers:
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    capa = DesiredCapabilities.CHROME
    capa["pageLoadStrategy"] = "none"
    driver = webdriver.Chrome(chrome_options=options, desired_capabilities=capa)
    driver.set_window_size(1440,900)
    driver.get('https://twitter.com/' + follower + '/photo')
    print('getting user #' + str(i) + "'s photo url!")
    time.sleep(15)
    
    plain_text = driver.page_source
    new_soup = BeautifulSoup(plain_text, 'lxml')
    pattern = re.compile('draggable="true" src="(.*).+?(?=\/>)')
    result = pattern.findall(str(new_soup))
    result = result[0]
    new_pattern = re.compile('(^.*.jpg)')
    new_result = new_pattern.findall(str(result))
    picture_urls.append(new_result)
    i+=1

# soup.select('.Table2__header-row') # Returns full results.

# len(soup.select('.Table2__header-row'))

C:\Users\sambe\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


getting user #1's photo url!
getting user #2's photo url!
getting user #3's photo url!
getting user #4's photo url!
getting user #5's photo url!
getting user #6's photo url!
getting user #7's photo url!
getting user #8's photo url!
getting user #9's photo url!
getting user #10's photo url!
getting user #11's photo url!
getting user #12's photo url!
getting user #13's photo url!
getting user #14's photo url!
getting user #15's photo url!
getting user #16's photo url!
getting user #17's photo url!
getting user #18's photo url!
getting user #19's photo url!
getting user #20's photo url!
getting user #21's photo url!
getting user #22's photo url!
getting user #23's photo url!
getting user #24's photo url!
getting user #25's photo url!
getting user #26's photo url!
getting user #27's photo url!
getting user #28's photo url!
getting user #29's photo url!
getting user #30's photo url!
getting user #31's photo url!
getting user #32's photo url!
getting user #33's photo url!
getting user #34's 

IndexError: list index out of range

In [107]:
picture_urls

[['https://pbs.twimg.com/profile_images/1215514540021932032/s1W81Kan_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/1232298147864510464/pZNpQVhR_400x400.jpg'],
 [],
 ['https://pbs.twimg.com/profile_images/1235491075546021889/-bwlS5UB_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/1273661017676042241/K1QX8Zii_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/624042272825278465/eWqIvP7H_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/1268192116318048256/74wibZ45_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/736255138130735104/UfLBjLvI_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/1173893581032980480/LH__2MJT_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/882632843339366401/8fgT5tV0_400x400.jpg'],
 [],
 ['https://pbs.twimg.com/profile_images/1273191976247201792/Ir_AZ_8K_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/1272644077130039299/t88dHMwM_400x400.jpg'],
 ['https://pbs.twimg.com/profile_images/1272606621705875457/sXElmmLk_

In [126]:
new_soup.prettify()

'<html>\n <head>\n </head>\n <body>\n </body>\n</html>'

In [91]:
pattern = re.compile('draggable="true" src="(.*).+?(?=\/>)')
result = pattern.findall(str(new_soup))
result = result[0]
new_pattern = re.compile('(^.*.jpg)')
new_result = new_pattern.findall(str(result))
print(new_result)

['https://pbs.twimg.com/profile_images/893494961366118400/Bi95bJ3r_400x400.jpg']


In [ ]:
for follower in my_followers:
    
        #buffer
        site = 'https://twitter.com/' + follower + '/photo'
        hdr = {'User-Agent': rand_user_agent()}
        req = Request(site,headers=hdr)
        page = urlopen(req)
        
        #notifications of progress
        print('scraping ' + site)
        
        #non-existing page 
        status_code = page.getcode()
        if status_code != 200:
            continue
        soup = BeautifulSoup(page)
        
        #Scrape summaries
        pic_urls = []
        
        scraped_summary = soup.findAll('src')
        if scraped_summary == []:
            temp_summary = 'null'
            pass
        else:
            scraped_summary = re.findall('b_expanded">(.*)\n.', str(scraped_summary))
            scraped_summary1 = re.findall("'(.*).r<br\/>\s", str(scraped_summary))
            try:
                temp_summary.append(scraped_summary1[0])
            except IndexError:
                pass
            scraped_summary2 = re.findall('\s<br\/>(.*)<\/span><s', str(scraped_summary))
            try:
                temp_summary.append(scraped_summary2[0])
            except IndexError:
                pass
            temp_summary = " ".join(temp_summary)
        summary.append(temp_summary)
        
        print("sleeping for " + str(nap) + " seconds")
        time.sleep(nap)
        print("awake!")
            
print('scraping complete!')

# Compile data into dataframe   
sampledf['summary'] = summary

https://twitter.com/Screen Name/photo